In [6]:
from dolfin import *
from PIL import Image, ImageOps
import numpy as np
from numpy import asarray

In [7]:
w = .1 # meters
h = .1 # meters
area_total = w * h

In [8]:
# image = Image.open('cat_posing.jpg')
image = Image.open('mehul.png')
im2 = ImageOps.grayscale(image)
brightness = asarray(im2)
total_brightness = sum(sum(brightness))

In [9]:
mesh_ = RectangleMesh.create([Point(0, 0), Point(w, h)],[512,512],CellType.Type.quadrilateral)

In [ ]:
for ii in range(4):
# Build function space with Lagrange multiplier
    P1 = FiniteElement("Lagrange", mesh_.ufl_cell(), 1)
    R = FiniteElement("Real", mesh_.ufl_cell(), 0)
    W = FunctionSpace(mesh_, P1 * R)

    # Define variational problem
    (u, c) = TrialFunction(W)
    (v, d) = TestFunctions(W)

    D = FunctionSpace(mesh_,"DG",0)
    br = Function(D)
    br.vector().vec().array[:] = brightness.reshape(mesh_.num_cells())

    V = FunctionSpace(mesh_,"CG",1)
    br= project(br,V)

    (cr,dr) = Function(W).split()
    (ar,er) = Function(W).split()

    cr.vector().vec().array[0:br.vector().size()] = br.vector()[:]

    area = Function(D)
    vv = TestFunction(D)

    area.vector()[:] = assemble(1*vv*dx)
    ar.vector().vec().array[0:area.vector().size()] = area.vector()[:]

    loss = cr / Constant(total_brightness) - ar / Constant(area_total)

    a = (inner(grad(u), grad(v)) + c*v + u*d)*dx
    L = -loss*v*dx

    # Compute solution
    w = Function(W)
    solve(a == L, w)
    (u, c) = w.split()

    with XDMFFile("phi.xdmf") as xdmf:
        xdmf.write(u,ii)

    print(norm(project(loss,D)))

    grad_u = project(grad(u),VectorFunctionSpace(mesh_,"CG",1))
    g_u = grad_u.compute_vertex_values()
    with XDMFFile("grad_u.xdmf") as xdmf:
        xdmf.write(grad_u)

    mesh_co=mesh_.coordinates()
    nv = mesh_.num_vertices()
    g_un=np.zeros((nv,2))
    g_un[:,0]=g_u[0:nv]
    g_un[:,1]=g_u[nv:2*nv]
    step = 10

    mesh_.coordinates()[:]=mesh_co+step*g_un

0.000279670266071706
0.00028082372854502263
0.00028196419148793944


In [96]:
g_u = grad_u.compute_vertex_values()

In [93]:
g_u[mesh_.num_vertices()]

3.747870501904941e-08

In [100]:
g_un

array([[ 2.64318478e-08,  3.74787050e-08],
       [ 8.36406075e-08,  3.88219996e-08],
       [ 1.82486202e-07,  4.04677279e-08],
       ...,
       [-7.90285747e-07,  4.17947883e-08],
       [-3.65399718e-07,  4.41713315e-08],
       [-1.16803704e-07,  4.73168635e-08]])

In [88]:
mesh_.coordinates()[0,0] = 0.0001

In [89]:
mesh_.coordinates()

array([[0.0001    , 0.        ],
       [0.00019531, 0.0001    ],
       [0.00039063, 0.        ],
       ...,
       [0.09960938, 0.1       ],
       [0.09980469, 0.1       ],
       [0.1       , 0.1       ]])

In [74]:
mesh.num_vertices()

263169

In [44]:
mag = np.sqrt(g_u[1][1]**2+g_u[1][1]**2)

In [45]:
g_u[1]/mag

array([0.46724043, 0.70710678])

In [52]:
mesh.hmin()

0.00016470458559087164

In [2]:
from PIL import Image
import numpy as np
from numpy import asarray

# load the image

image = Image.open('cat_posing.jpg')

# convert image to numpy array

data = asarray(image)

print(type(data))

# summarize shape

print(data.shape)



# create Pillow image

image2 = Image.fromarray(data)

print(type(image2))


# summarize image details

print(image2.mode)

print(image2.size)

<class 'numpy.ndarray'>
(512, 512, 3)
<class 'PIL.Image.Image'>
RGB
(512, 512)


In [3]:
print(data)

[[[236 224 172]
  [236 224 172]
  [236 224 172]
  ...
  [232 230 189]
  [230 231 189]
  [230 231 189]]

 [[236 224 172]
  [236 224 172]
  [236 224 172]
  ...
  [232 230 189]
  [230 231 189]
  [230 231 189]]

 [[236 224 172]
  [236 224 172]
  [236 224 172]
  ...
  [232 230 189]
  [230 231 189]
  [230 231 189]]

 ...

 [[178 136  78]
  [178 136  78]
  [180 138  80]
  ...
  [234 230 192]
  [234 230 192]
  [234 230 192]]

 [[168 126  68]
  [172 130  72]
  [185 143  85]
  ...
  [234 230 192]
  [234 230 192]
  [234 230 192]]

 [[145 100  43]
  [151 109  51]
  [142  99  44]
  ...
  [234 230 192]
  [234 230 192]
  [234 230 192]]]


In [4]:
# Importing Image and ImageOps module from PIL package
from PIL import Image, ImageOps


# applying grayscale method
im2 = ImageOps.grayscale(image)
brightness = asarray(im2)

In [6]:
total_brightness = sum(sum(brightness))

In [7]:
normalized_brightness= np.divide(brightness,total_brightness)

In [9]:
from dolfin import *

In [10]:
mesh = RectangleMesh(Point(0,0), Point(w,h), 512,512)

In [11]:
mesh = RectangleMesh.create([Point(0,0),Point(w,h)],[512,512],CellType.Type.quadrilateral)

In [12]:
V = FunctionSpace(mesh,"CG",1)
D = FunctionSpace(mesh,"DG",0)

Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.


In [13]:
br = Function(D)

br.vector().vec().array[:] = brightness.reshape(mesh.num_cells())

In [14]:
with XDMFFile("brightness.xdmf") as xdmf:
    xdmf.write(br)

In [15]:
area = Function(D)
v = TestFunction(D)

area.vector()[:] = assemble(1*v*dx)

Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.


In [16]:
area.vector()[:].sum()

0.009999999999999966

In [17]:
loss = br / Constant(total_brightness) - area / Constant(area_total)

In [18]:
loss = project(loss,V)

Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.


In [19]:
loss

Coefficient(FunctionSpace(Mesh(VectorElement(FiniteElement('Q', quadrilateral, 1), dim=2), 6), FiniteElement('Q', quadrilateral, 1)), 32)

In [20]:
with XDMFFile("loss.xdmf") as xdmf:
    xdmf.write(loss)

In [21]:
# gloss = project(grad(loss),VectorFunctionSpace(mesh,"DG",0))

with XDMFFile("gloss.xdmf") as xdmf:
    xdmf.write(gloss)

In [23]:
ph =  Function(D)

In [24]:
u = TrialFunction(D)
v = TestFunction(D)

a = inner(grad(u), grad(v))*dx

In [25]:
solve(a == -loss*v*dx,ph)

Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.


In [158]:
with XDMFFile("phi.xdmf") as xdmf:
    xdmf.write(ph)

In [159]:
ph.vector().min()

-inf

In [160]:
ph.vector().max()

-inf